In [1]:
import numpy as np
import random
import os
import math

from glob import glob
import pandas as pd
import cv2

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def get_train_data(data_dir):
    meta_path_list = []
    img_path_list = []
    label_list = pd.DataFrame({
                  'img_name':[],'leaf_weight':[],
                  })

    for case_name in os.listdir(data_dir):
        current_path = os.path.join(data_dir, case_name)
        if os.path.isdir(current_path):
            # get image path
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.jpg')))
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.png')))
            
            # get label
            label_df = pd.read_csv(current_path+'/label.csv')
            label_list=pd.concat([label_list,label_df])
            
            # get meta path
            meta_path_list.extend(glob(os.path.join(current_path, 'meta', '*.csv')))
                
    return img_path_list,label_list,meta_path_list

def get_test_data(data_dir):
    
    # get image path
    img_path_list = glob(os.path.join(data_dir, 'image', '*.jpg'))
    img_path_list.extend(glob(os.path.join(data_dir, 'image', '*.png')))
    img_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))
    
    # get meta path
    meta_path_list = glob(os.path.join(data_dir, 'meta', '*.csv'))
    meta_path_list.sort(key=lambda x:int(x.split('/')[-1].split('.')[0]))
    return img_path_list,meta_path_list

In [4]:
all_img_path, LABEL, all_meta_path = get_train_data('./dataset/train')
test_img_path, test_meta_path = get_test_data('./dataset/test')

In [5]:
LABEL['img_name'] = LABEL['img_name'].apply(lambda x:x[:-4])

In [6]:
META = pd.DataFrame({
                  'img_name':[], 
                  'day_in_temp':[],'night_in_temp':[],
                  'day_out_temp':[],'night_out_temp':[], #추가
                  'day_in_humi':[],'night_in_humi':[],
                  'day_out_humi':[],'night_out_humi':[], #추가
                  'day_co2':[],'night_co2':[],
                  'day_ec':[],'night_ec':[],   
                  'day_water':[],'night_water':[],   
                  'day_white':[],'night_white':[],   
                  'day_red':[],'night_red':[],    
                  'day_blue':[],'night_blue':[],
                  'day_cool_temp':[],'night_cool_temp':[],
                  'day_cooling':[],'night_cooling':[],
                  'day_heat_temp':[],'night_heat_temp':[],
                  'day_heating':[],'night_heating':[],
                  'day_light':[], 'night_light':[],
                  'day_white_light':[], 'night_white_light':[],
                  'day_red_light':[], 'night_red_light':[],
                  'day_blue_light':[], 'night_blue_light':[],
   


                  })
IMG = pd.DataFrame({
                  'img_name':[],'img_pixel':[],
                  })


for data_dir in all_img_path:
    
    #영상 읽어서 그레이 스케일로 변환
    img = cv2.imread(data_dir)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 8x8 크기로 축소 ---①
    gray = cv2.resize(gray, (150,150))
    # 영상의 평균값 구하기 ---②
    avg = gray.mean()
    # 평균값을 기준으로 0과 1로 변환 ---③
    bin = 1 * (gray > avg)
    #print(bin)

    result = 0

    for i in bin.reshape(-1,) :
        if(int(i)==0):
            result+=1

    
    IMG = IMG.append({
                  'img_name':data_dir[-13:-4],'img_pixel':result,
                  }, ignore_index=True)
    
    
    
 


for data_dir in all_meta_path:
    meta = pd.read_csv(data_dir)
    re_meta = meta[meta.columns[1:]]
    
    re_meta = re_meta.fillna(0)
    
    result = []
    
    for name in meta.columns[1:] :
        s1 = re_meta[name][0:314].sum()
        s2 = re_meta[name][974:1340].sum()

        s3 = re_meta[name][314:974].sum()

        day_mean = (s1+s2) / 680
        night_mean = s3 / 660

        result.append(day_mean)
        result.append(night_mean)
        
    META = META.append({
          
                  'img_name':data_dir[-13:-4],
                  'day_in_temp':result[0],'night_in_temp':result[1],
                  'day_out_temp':result[2],'night_out_temp':result[3], #추가
                  'day_in_humi':result[4],'night_in_humi':result[5],
                  'day_out_humi':result[6],'night_out_humi':result[7], #추가
                  'day_co2':result[8],'night_co2':result[9],
                  'day_ec':result[10],'night_ec':result[11],   
                  'day_water':result[12],'night_water':result[13],   
                  'day_white':result[14],'night_white':result[15],   
                  'day_red':result[16],'night_red':result[17],    
                  'day_blue':result[18],'night_blue':result[19],
                  'day_cool_temp':result[20],'night_cool_temp':result[21],
                  'day_cooling':result[22],'night_cooling':result[23],
                  'day_heat_temp':result[24],'night_heat_temp':result[25],
                  'day_heating':result[26],'night_heating':result[27],
                  'day_light':result[28], 'night_light':result[29],
                  'day_white_light':result[30], 'night_white_light':result[31],
                  'day_red_light':result[32], 'night_red_light':result[33],
                  'day_blue_light':result[34], 'night_blue_light':result[35],

                  }, ignore_index=True)

In [7]:
META.head()

,img_name,day_in_temp,night_in_temp,day_out_temp,night_out_temp,day_in_humi,night_in_humi,day_out_humi,night_out_humi,day_co2,...,day_heating,night_heating,day_light,night_light,day_white_light,night_white_light,day_red_light,night_red_light,day_blue_light,night_blue_light
0,CASE68_20,27.116618,28.453939,26.944265,29.053333,53.342941,54.650454,17.095147,13.524697,509.450000,...,0.000000,0.0,151.497146,119.216611,151.497146,119.216611,0.0,0.0,0.0,0.0
1,CASE68_12,28.505441,31.693788,26.758529,30.385758,51.913088,48.921364,19.667206,13.650758,494.114706,...,0.000000,0.0,151.092183,119.530709,151.092183,119.530709,0.0,0.0,0.0,0.0
2,CASE68_06,27.910441,27.757576,26.878529,26.980909,43.995000,41.884697,24.204265,21.200000,465.029412,...,0.000000,0.0,151.533547,119.305683,151.533547,119.305683,0.0,0.0,0.0,0.0
3,CASE68_23,23.798823,24.180757,20.639264,21.901060,52.798823,53.071515,27.589118,25.566970,470.408824,...,1.556471,0.0,151.620000,119.216611,151.620000,119.216611,0.0,0.0,0.0,0.0
4,CASE68_14,26.723088,32.251212,24.887059,33.318182,52.098235,49.665303,23.418529,13.886364,496.963235,...,0.000000,0.0,151.460745,119.329123,151.460745,119.329123,0.0,0.0,0.0,0.0


In [8]:
IMG.head()

,img_name,img_pixel
0,CASE68_14,7555.0
1,CASE68_26,10995.0
2,CASE68_22,9348.0
3,CASE68_08,7154.0
4,CASE68_09,7140.0


In [9]:
LABEL.head()

,img_name,leaf_weight
0,CASE68_01,1.736
1,CASE68_02,2.372
2,CASE68_03,2.802
3,CASE68_04,3.730
4,CASE68_05,4.531


In [10]:
#META.to_csv("./train_meta.csv",index=False)

In [11]:
TEST_META = pd.DataFrame({
                  'img_name':[], 
                  'day_in_temp':[],'night_in_temp':[],
                  'day_out_temp':[],'night_out_temp':[], #추가
                  'day_in_humi':[],'night_in_humi':[],
                  'day_out_humi':[],'night_out_humi':[], #추가
                  'day_co2':[],'night_co2':[],
                  'day_ec':[],'night_ec':[],   
                  'day_water':[],'night_water':[],   
                  'day_white':[],'night_white':[],   
                  'day_red':[],'night_red':[],    
                  'day_blue':[],'night_blue':[],
                  'day_cool_temp':[],'night_cool_temp':[],
                  'day_cooling':[],'night_cooling':[],
                  'day_heat_temp':[],'night_heat_temp':[],
                  'day_heating':[],'night_heating':[],
                  'day_light':[], 'night_light':[],
                  'day_white_light':[], 'night_white_light':[],
                  'day_red_light':[], 'night_red_light':[],
                  'day_blue_light':[], 'night_blue_light':[],
   
                  }) 


TEST_IMG = pd.DataFrame({
                  'img_name':[],'img_pixel':[],
                  })


for data_dir in test_img_path:
    
    #영상 읽어서 그레이 스케일로 변환
    img = cv2.imread(data_dir)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 8x8 크기로 축소 ---①
    gray = cv2.resize(gray, (150,150))
    # 영상의 평균값 구하기 ---②
    avg = gray.mean()
    # 평균값을 기준으로 0과 1로 변환 ---③
    bin = 1 * (gray > avg)
    #print(bin)

    result = 0

    for i in bin.reshape(-1,) :
        if(int(i)==0):
            result+=1

    
    TEST_IMG = TEST_IMG.append({
                  'img_name':data_dir[-7:-4],'img_pixel':result,
                  }, ignore_index=True)
    
    
#col_name = ['내부온도관측치','내부습도관측치','CO2관측치','EC관측치','최근분무량','화이트 LED동작강도','레드 LED동작강도','블루 LED동작강도']

for data_dir in test_meta_path:
    meta = pd.read_csv(data_dir)
    re_meta = meta[meta.columns[1:]]
    
    re_meta = re_meta.fillna(0)
    
    result = []
    
    for name in meta.columns[1:]:
        s1 = re_meta[name][0:314].sum()
        s2 = re_meta[name][974:1340].sum()

        s3 = re_meta[name][314:974].sum()

        day_mean = (s1+s2) / 680
        night_mean = s3 / 660

        result.append(day_mean)
        result.append(night_mean)
        
    TEST_META = TEST_META.append({
          
                  'img_name':data_dir[-7:-4],
                  'day_in_temp':result[0],'night_in_temp':result[1],
                  'day_out_temp':result[2],'night_out_temp':result[3], #추가
                  'day_in_humi':result[4],'night_in_humi':result[5],
                  'day_out_humi':result[6],'night_out_humi':result[7], #추가
                  'day_co2':result[8],'night_co2':result[9],
                  'day_ec':result[10],'night_ec':result[11],   
                  'day_water':result[12],'night_water':result[13],   
                  'day_white':result[14],'night_white':result[15],   
                  'day_red':result[16],'night_red':result[17],    
                  'day_blue':result[18],'night_blue':result[19],
                  'day_cool_temp':result[20],'night_cool_temp':result[21],
                  'day_cooling':result[22],'night_cooling':result[23],
                  'day_heat_temp':result[24],'night_heat_temp':result[25],
                  'day_heating':result[26],'night_heating':result[27],
                  'day_light':result[28], 'night_light':result[29],
                  'day_white_light':result[30], 'night_white_light':result[31],
                  'day_red_light':result[32], 'night_red_light':result[33],
                  'day_blue_light':result[34], 'night_blue_light':result[35],

                  }, ignore_index=True)

In [12]:
TEST_META.head()

,img_name,day_in_temp,night_in_temp,day_out_temp,night_out_temp,day_in_humi,night_in_humi,day_out_humi,night_out_humi,day_co2,...,day_heating,night_heating,day_light,night_light,day_white_light,night_white_light,day_red_light,night_red_light,day_blue_light,night_blue_light
0,001,23.603971,23.449394,22.509853,22.641515,68.180294,73.767879,35.335441,34.069394,393.976471,...,0.0,0.000000,183.647515,149.009827,151.219587,115.316169,9.782788,14.316527,22.645140,19.377130
1,002,27.546912,26.655303,29.152353,28.186970,64.208529,63.803182,49.485735,53.207576,496.073529,...,0.0,0.000000,175.809137,50.939302,143.620842,41.812542,21.870129,6.240602,10.318167,2.886158
2,003,24.757353,27.445757,25.466618,25.601667,81.508382,76.193788,65.603677,65.574697,512.982353,...,0.0,1.304918,28.311234,355.695686,25.826635,286.424587,1.275169,25.238207,1.209430,44.032891
3,004,23.929412,21.572879,22.430588,22.283030,69.002647,74.375152,45.251765,45.541061,382.332353,...,0.0,0.000000,251.846902,66.711633,201.917326,59.847395,17.910776,3.071409,32.018799,3.792829
4,005,21.073529,25.335000,22.397647,22.686212,76.088235,70.994697,33.605441,33.897273,446.497059,...,0.0,0.000000,27.867698,356.466638,25.444422,287.099664,1.248401,25.255758,1.174875,44.111216


In [13]:
TEST_IMG.head()

,img_name,img_pixel
0,001,8596.0
1,002,12828.0
2,003,7357.0
3,004,8367.0
4,005,8478.0


In [14]:
#TEST_META.to_csv("./test_meta.csv",index=False)

In [15]:
META = META.sort_values(by=['img_name'],ignore_index=True)
IMG = IMG.sort_values(by=['img_name'],ignore_index=True)
TEST_META = TEST_META.sort_values(by=['img_name'],ignore_index=True)
TEST_IMG = TEST_IMG.sort_values(by=['img_name'],ignore_index=True)
LABEL = LABEL.sort_values(by=['img_name'],ignore_index=True)

In [16]:
TEST_META.head()

,img_name,day_in_temp,night_in_temp,day_out_temp,night_out_temp,day_in_humi,night_in_humi,day_out_humi,night_out_humi,day_co2,...,day_heating,night_heating,day_light,night_light,day_white_light,night_white_light,day_red_light,night_red_light,day_blue_light,night_blue_light
0,001,23.603971,23.449394,22.509853,22.641515,68.180294,73.767879,35.335441,34.069394,393.976471,...,0.0,0.000000,183.647515,149.009827,151.219587,115.316169,9.782788,14.316527,22.645140,19.377130
1,002,27.546912,26.655303,29.152353,28.186970,64.208529,63.803182,49.485735,53.207576,496.073529,...,0.0,0.000000,175.809137,50.939302,143.620842,41.812542,21.870129,6.240602,10.318167,2.886158
2,003,24.757353,27.445757,25.466618,25.601667,81.508382,76.193788,65.603677,65.574697,512.982353,...,0.0,1.304918,28.311234,355.695686,25.826635,286.424587,1.275169,25.238207,1.209430,44.032891
3,004,23.929412,21.572879,22.430588,22.283030,69.002647,74.375152,45.251765,45.541061,382.332353,...,0.0,0.000000,251.846902,66.711633,201.917326,59.847395,17.910776,3.071409,32.018799,3.792829
4,005,21.073529,25.335000,22.397647,22.686212,76.088235,70.994697,33.605441,33.897273,446.497059,...,0.0,0.000000,27.867698,356.466638,25.444422,287.099664,1.248401,25.255758,1.174875,44.111216


In [17]:
TEST_IMG.head()

,img_name,img_pixel
0,001,8596.0
1,002,12828.0
2,003,7357.0
3,004,8367.0
4,005,8478.0


In [18]:
IMG

,img_name,img_pixel
0,CASE01_01,7777.0
1,CASE01_02,8004.0
2,CASE01_03,8336.0
3,CASE01_04,8581.0
4,CASE01_05,8969.0
...,...,...
1587,CASE74_25,10765.0
1588,CASE74_26,10775.0
1589,CASE74_27,10808.0
1590,CASE75_01,10698.0


In [19]:
META

,img_name,day_in_temp,night_in_temp,day_out_temp,night_out_temp,day_in_humi,night_in_humi,day_out_humi,night_out_humi,day_co2,...,day_heating,night_heating,day_light,night_light,day_white_light,night_white_light,day_red_light,night_red_light,day_blue_light,night_blue_light
0,CASE01_01,18.625882,26.629242,42.220588,44.986970,78.413088,76.605303,2.660294,5.481818,559.897059,...,12.045657,8.201072,25.772033,286.616797,1.263002,25.295875,1.197912,44.082734,0.000000,0.000000
1,CASE01_02,18.971176,27.916212,52.066765,35.233636,80.035735,74.764091,1.775000,6.395455,563.860294,...,13.748063,13.673527,25.703781,286.602733,1.248401,25.295875,1.184090,44.035264,0.000000,0.000000
2,CASE01_03,19.191912,27.503182,47.165588,30.250455,80.032647,74.511970,2.661765,5.786364,569.777941,...,14.833258,11.733555,25.790234,286.752750,1.243534,25.270802,1.174875,44.139698,0.000000,0.000000
3,CASE01_04,18.338971,24.238939,59.114559,56.608939,80.903382,78.914545,1.773529,4.872727,560.339706,...,10.636027,3.607754,25.644629,286.827758,1.255701,25.245729,1.188697,44.104095,0.000000,0.000000
4,CASE01_05,19.151765,25.195454,67.153970,68.003636,82.164706,80.358939,1.773529,5.786364,565.273529,...,14.640814,6.560112,25.612778,286.846510,1.270302,25.288353,1.202519,44.087481,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,CASE74_25,23.309559,27.401818,22.734853,22.808485,80.478823,79.621515,56.503529,55.913182,559.838235,...,0.000000,0.000000,28.252212,356.096615,25.767483,286.766814,1.277603,25.270802,1.207126,44.058999
1588,CASE74_26,23.433823,27.644848,23.207794,23.657576,79.814265,79.622576,53.010147,53.508485,566.097059,...,0.000000,0.000000,28.346080,355.765426,25.844835,286.448028,1.284904,25.260773,1.216341,44.056626
1589,CASE74_27,23.315000,27.305909,22.738970,22.527424,79.597794,78.207727,48.744559,47.848485,694.885294,...,0.000000,0.000000,27.758553,356.345702,25.389821,286.987151,1.219198,25.275816,1.149535,44.082734
1590,CASE75_01,22.890294,27.281970,21.757500,22.558636,80.480147,79.483788,56.072794,57.914697,567.451471,...,0.000000,0.000000,28.045024,356.190952,25.621878,286.785566,1.245967,25.308411,1.177179,44.096975


In [20]:
LABEL.head()

,img_name,leaf_weight
0,CASE01_01,49.193
1,CASE01_02,59.764
2,CASE01_03,72.209
3,CASE01_04,85.737
4,CASE01_05,102.537


In [21]:
TRAIN_META = pd.concat([META,IMG['img_pixel']],axis=1)
TRAIN_META = pd.concat([TRAIN_META,LABEL['leaf_weight']],axis=1)

In [22]:
TRAIN_META

,img_name,day_in_temp,night_in_temp,day_out_temp,night_out_temp,day_in_humi,night_in_humi,day_out_humi,night_out_humi,day_co2,...,day_light,night_light,day_white_light,night_white_light,day_red_light,night_red_light,day_blue_light,night_blue_light,img_pixel,leaf_weight
0,CASE01_01,18.625882,26.629242,42.220588,44.986970,78.413088,76.605303,2.660294,5.481818,559.897059,...,25.772033,286.616797,1.263002,25.295875,1.197912,44.082734,0.000000,0.000000,7777.0,49.193
1,CASE01_02,18.971176,27.916212,52.066765,35.233636,80.035735,74.764091,1.775000,6.395455,563.860294,...,25.703781,286.602733,1.248401,25.295875,1.184090,44.035264,0.000000,0.000000,8004.0,59.764
2,CASE01_03,19.191912,27.503182,47.165588,30.250455,80.032647,74.511970,2.661765,5.786364,569.777941,...,25.790234,286.752750,1.243534,25.270802,1.174875,44.139698,0.000000,0.000000,8336.0,72.209
3,CASE01_04,18.338971,24.238939,59.114559,56.608939,80.903382,78.914545,1.773529,4.872727,560.339706,...,25.644629,286.827758,1.255701,25.245729,1.188697,44.104095,0.000000,0.000000,8581.0,85.737
4,CASE01_05,19.151765,25.195454,67.153970,68.003636,82.164706,80.358939,1.773529,5.786364,565.273529,...,25.612778,286.846510,1.270302,25.288353,1.202519,44.087481,0.000000,0.000000,8969.0,102.537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,CASE74_25,23.309559,27.401818,22.734853,22.808485,80.478823,79.621515,56.503529,55.913182,559.838235,...,28.252212,356.096615,25.767483,286.766814,1.277603,25.270802,1.207126,44.058999,10765.0,211.497
1588,CASE74_26,23.433823,27.644848,23.207794,23.657576,79.814265,79.622576,53.010147,53.508485,566.097059,...,28.346080,355.765426,25.844835,286.448028,1.284904,25.260773,1.216341,44.056626,10775.0,214.116
1589,CASE74_27,23.315000,27.305909,22.738970,22.527424,79.597794,78.207727,48.744559,47.848485,694.885294,...,27.758553,356.345702,25.389821,286.987151,1.219198,25.275816,1.149535,44.082734,10808.0,214.293
1590,CASE75_01,22.890294,27.281970,21.757500,22.558636,80.480147,79.483788,56.072794,57.914697,567.451471,...,28.045024,356.190952,25.621878,286.785566,1.245967,25.308411,1.177179,44.096975,10698.0,210.872


In [23]:
TEST_META = pd.concat([TEST_META,TEST_IMG['img_pixel']],axis=1)

In [24]:
TEST_META

,img_name,day_in_temp,night_in_temp,day_out_temp,night_out_temp,day_in_humi,night_in_humi,day_out_humi,night_out_humi,day_co2,...,night_heating,day_light,night_light,day_white_light,night_white_light,day_red_light,night_red_light,day_blue_light,night_blue_light,img_pixel
0,001,23.603971,23.449394,22.509853,22.641515,68.180294,73.767879,35.335441,34.069394,393.976471,...,0.000000,183.647515,149.009827,151.219587,115.316169,9.782788,14.316527,22.645140,19.377130,8596.0
1,002,27.546912,26.655303,29.152353,28.186970,64.208529,63.803182,49.485735,53.207576,496.073529,...,0.000000,175.809137,50.939302,143.620842,41.812542,21.870129,6.240602,10.318167,2.886158,12828.0
2,003,24.757353,27.445757,25.466618,25.601667,81.508382,76.193788,65.603677,65.574697,512.982353,...,1.304918,28.311234,355.695686,25.826635,286.424587,1.275169,25.238207,1.209430,44.032891,7357.0
3,004,23.929412,21.572879,22.430588,22.283030,69.002647,74.375152,45.251765,45.541061,382.332353,...,0.000000,251.846902,66.711633,201.917326,59.847395,17.910776,3.071409,32.018799,3.792829,8367.0
4,005,21.073529,25.335000,22.397647,22.686212,76.088235,70.994697,33.605441,33.897273,446.497059,...,0.000000,27.867698,356.466638,25.444422,287.099664,1.248401,25.255758,1.174875,44.111216,8478.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,456,27.383088,25.804242,28.820294,27.523182,59.682059,59.565152,44.732647,47.116818,478.552941,...,0.000000,175.744975,51.100579,143.611741,41.939119,21.831192,6.268182,10.302041,2.893278,8355.0
456,457,22.477794,25.836364,23.465735,24.251970,70.557647,67.142121,28.046029,29.564848,411.600000,...,0.000000,142.387624,471.995277,107.060410,304.253167,32.730971,150.448900,2.596243,17.293211,8617.0
457,458,22.100735,25.600758,22.565882,22.686970,83.253529,77.547121,51.103823,51.594242,431.417647,...,0.000000,27.719904,356.400371,25.289717,287.034031,1.248401,25.288353,1.181786,44.077987,8281.0
458,459,22.071471,25.578030,22.522500,22.660303,77.380882,72.682727,34.023971,38.908788,414.801471,...,0.000000,27.906517,356.314991,25.471723,286.954335,1.248401,25.270802,1.186393,44.089855,8547.0


In [25]:
TRAIN_META.to_csv("./train_V2.csv",index=False)
TEST_META.to_csv("./test_V2.csv",index=False)